In [61]:
import os
import pickle
import numpy as np

In [62]:
data_path = 'data'
train_origin_data = 'data.orig'
annotated_data = 'Train_output'
entity_type_list = ["d", "s", "c", "i", "a", "b", "t", "p"]
all_data = ["raw_data.orig"]

In [63]:
def resolve_line(line_num, origin_line, line, entity_type_list):
    words = []
    tags = []
    for part in line.split(" "):
        part = part.strip("\n")
        if len(part) >= 2 and part[-2] == '\\' and part[-1] in entity_type_list:
            words.extend(list(part[ :-2]))
            part_tag = ["B-" + part[-1].upper()] + ["I-" + part[-1].upper()] * (len(part) - 3)
            tags.extend(part_tag)
        else:
            words.extend(list(part))
            part_tag = ["O"] * len(part)
            tags.extend(part_tag)
    # print(len(words))
    # print(len(tags))
    assert len(words) == len(tags)
    if(list(origin_line.strip()) != words):
        print("At line {} origin and annotated line don't match! ".format(line_num))
    assert words == list(origin_line.strip())
    return words, tags

In [64]:
def read_corpora(data_path, train_origin_data,train_annotated_data,entity_type_list):
    data_origin = os.path.join(data_path, train_origin_data)
    annotation = os.path.join(data_path, train_annotated_data)
    with open(data_origin) as f1:
        origin_lines = f1.readlines()
    with open(annotation) as f2:
        annotated_lines = f2.readlines()
    assert len(origin_lines) == len(annotated_lines)
    f1.close()
    f2.close()
    train_data = []
    for i in range(len(origin_lines)):
        # cache1 = origin_lines[i]
        # cache2 = annotated_lines[i]
        # print(f'length origin lines:{len(cache1)}')
        # print(f'length annotated lines:{len(cache2)}')
        # break
        words, tags = resolve_line(i+1, origin_lines[i], annotated_lines[i], entity_type_list)
        train_data.append((words, tags))
    return train_data

In [78]:
def calculate_accuracy(re_true,_predict):
    test_acc = 0.0
    cnt = 0.0
    for i in range(len(re_true)):
            if re_true[i] == _predict[i]:
                    cnt += 1
            else:
                    pass
    test_acc = cnt/len(re_true)
    return test_acc

In [91]:
pre_data = read_corpora(data_path, train_origin_data,annotated_data, entity_type_list)
# print('Data is preprocessed!')
_predict = []

for i in range(len(pre_data)):
        _predict.append(pre_data[i][1])



groud_truth = 'Ground_truth'
groud_truth = os.path.join(data_path, groud_truth)
with open(groud_truth) as f:
        _true_labels = f.readlines()

rows = len(_true_labels)
re_true = []
for i in range(rows):
        cache = _true_labels[i].strip().split(' ')
        re_true.append(cache)


# print('Total true labels:',len(re_true))
# print('length of 1st true labels:',len(re_true[0]))
# print('Total pre labels:',len(pre_data))
# print('Total pre labels:',len(_predict))
# print('length of 1st pre labels:',len(_predict[0]))

# print('1st pre labels:',_predict[0])
# print('1st true labels:',re_true[0])

test_acc = calculate_accuracy(re_true[0],_predict[0])
print(f'test_acc:{test_acc}')


test_acc:0.8797653958944281


In [110]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score,recall_score,f1_score
from sklearn.preprocessing import MultiLabelBinarizer  

acc = 0.0
precision = 0.0
recall = 0.0
f1 = 0.0
for i in range(rows):


    acc += calculate_accuracy(re_true[i],_predict[i])/rows
    precision += precision_score(re_true[i],_predict[i],average='macro')/rows
    recall += recall_score(re_true[i],_predict[i],average='macro')/rows
    f1 += f1_score(re_true[i],_predict[i],average='macro')/rows


mat = confusion_matrix(np.array(re_true[0]),np.array(_predict[0]))


print(f'acc:{acc}')
print(f'precision:{precision}')
print(f'recall:{recall}')
print(f'f1:{f1}')

print(f'mat:{mat}')

acc:0.7593457558631598
precision:0.6743578777264214
recall:0.7367999857253221
f1:0.6812175802171195
mat:[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   1   0   0   0   0   0   1   0   0   0   0   0   0]
 [  0   0   7   0   0   1   0   0   1   0   0   0   0   0]
 [  0   0   0   3   0   0   0   0   0   0   0   0   0   0]
 [  1   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0  10   0   0   0   0   0   2   0   0]
 [  0   0   0   0   0   0   4   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0  13   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0  40   0   0   3   0   0]
 [  0   0   0   0   0   0   0   0   0  19   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   1]
 [  0   0   0   0   0   0   0   0   0   0   0  36   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0  15   0]
 [  0   1   2   2   0   2   0   0   7  10   0   7   0 152]]
